<a href="https://colab.research.google.com/github/nilsroemertum/synlab-slim-viewer/blob/master/notebooks/viewers_deployment/slim_Firebase_deployment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Slim Deployment with Firebase and Google Cloud

This notebook complements the tutorial available in this document:
[Google Doc: Slim Deployment with Firebase and Google Cloud](https://tinyurl.com/idc-slim-gcp).
You need to complete the steps discussed in that document before proceeding with this notebook! Please use the document above for providing feedback or asking questions.



# Prerequisites

> **User interaction required**: In the following cell, set the variable gcpProjectID to your **Project ID**, gcpClientID to your **Client ID** and FirebaseProjectName to the **name of your Firebase project**.

In [17]:
gcpProjectID = 'synlab-pacs-workflow'
gcpClientID = '"62039621046-rt1fu02rnnfrlf2icjh2q1cmervka8au.apps.googleusercontent.com"'
FirebaseProjectName = 'synlab-pacs-slim-viewer'

## Install yarn

In [2]:
!npm install -g yarn

⠙⠹⠸⠼⠴⠦⠧⠇
added 1 package in 1s
⠇

## Install Firebase command line tools

In [3]:
!curl -sL https://firebase.tools | bash

-- Checking for existing firebase-tools...
-- Checking your machine type...
-- Downloading binary from https://firebase.tools/bin/linux/latest
######################################################################## 100.0%
-- Setting permissions on binary... /usr/local/bin/firebase
-- Checking your PATH variable...
-- firebase-tools@14.16.0 is now installed
-- All Done!


## Clone Slim source code

In [4]:
!git clone https://github.com/nilsroemertum/synlab-slim-viewer.git

Cloning into 'synlab-slim-viewer'...
remote: Enumerating objects: 1901, done.
remote: Counting objects: 100% (1090/1090), done.
remote: Compressing objects: 100% (364/364), done.
remote: Total 1901 (delta 920), reused 723 (delta 723), pack-reused 811 (from 2)
Receiving objects: 100% (1901/1901), 33.83 MiB | 21.27 MiB/s, done.
Resolving deltas: 100% (1193/1193), done.


## Install the node version management library (n) to update the node version to match Slim's

In [5]:
!npm install -g n

⠙⠹⠸⠼
added 1 package in 826ms
⠼

## Install and use node 14 using n



In [6]:
!n install 24 && n 24

  installing : node-v24.8.0
       mkdir : /usr/local/n/versions/node/24.8.0
       fetch : https://nodejs.org/dist/v24.8.0/node-v24.8.0-linux-x64.tar.xz
######################################################################## 100.0%
     copying : node/24.8.0
   installed : v24.8.0 (with npm 11.6.0)

Note: the node command changed location and the old location may be remembered in your current shell.
         old : /tools/node/bin/node
         new : /usr/local/bin/node
If "node --version" shows the old version then start a new shell, or reset the location hash with:
hash -r  (for bash, zsh, ash, dash, and ksh)
rehash   (for csh and tcsh)
     copying : node/24.8.0
   installed : v24.8.0 (with npm 11.6.0)


# Configure and deploy Slim using Firebase


## Install Slim's dependencies using yarn



In [7]:
!cd synlab-slim-viewer && yarn

yarn install v1.22.22
[1/4] Resolving packages...
warning Resolution field "semver@7.5.2" is incompatible with requested version "semver@^7.6.0"
warning Resolution field "semver@7.5.2" is incompatible with requested version "semver@^7.5.3"
warning Resolution field "wrap-ansi@7.0.0" is incompatible with requested version "wrap-ansi@^8.1.0"
warning Resolution field "make-dir@3.1.0" is incompatible with requested version "make-dir@^2.1.0"
warning Resolution field "rollup@3.29.5" is incompatible with requested version "rollup@^2.43.1"
warning Resolution field "nth-check@2.0.1" is incompatible with requested version "nth-check@^1.0.2"
[2/4] Fetching packages...
[] 0/1728[] 6/1728[] 10/1728[] 14/1728[] 18/1728[] 24/1728[] 33/1728[] 39/1728[] 45/1728[] 52/1728[] 59/1728[] 67/1728[] 71/1728[] 77/1728[] 83/1728[] 87/1728[] 91/1728[] 96/1728[] 100/1728[] 104/1728[] 107/1728[] 110/1728[] 115/1728[] 120/1728[] 124/1728[] 128/1728[] 132/1728[] 136/1728[] 138/1728[] 142/1728[] 145/1728[] 147/1728[] 

## Update Slim's configuration file
> The following code snipped configures things like OpenID Connect (OIDC) and server settings.



In [18]:
settings = """
window.config = {
  // This must match the location configured for web server
  path: '/',
  servers: [
    {
      id: 'example',
      // This must match the proxy location configured for the DICOMweb server
      url:"https://proxy.imaging.datacommons.cancer.gov/current/viewer-only-no-downloads-see-tinyurl-dot-com-slash-3j3d9jyp/dicomWeb",
      write: false
    }
  ],
 oidc: {
    authority: "https://accounts.google.com",
    clientId: %s,
    scope: "email profile openid https://www.googleapis.com/auth/cloud-healthcare",
    grantType: "implicit"
  },
  disableWorklist: false,
  disableAnnotationTools: false,
  enableServerSelection: true,
  mode: 'light',
  preload: true,
  annotations: [
    {
      finding: {
        value: '85756007',
        schemeDesignator: 'SCT',
        meaning: 'Tissue'
      },
      findingCategory: {
        value: '91723000',
        schemeDesignator: 'SCT',
        meaning: 'Anatomical structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 255, 0, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      },
    },
    {
      finding: {
        value: '108369006',
        schemeDesignator: 'SCT',
        meaning: 'Tumor'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 0, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '34823008',
        schemeDesignator: 'SCT',
        meaning: 'Tumor necrosis'
      },
      findingCategory: {
        value: '49755003',
        schemeDesignator: 'SCT',
        meaning: 'Morphologically abnormal structure'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 51, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '369705002',
        schemeDesignator: 'SCT',
        meaning: 'Invasive tumor border'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['line', 'freehandline'],
      style: {
        stroke: {
          color: [51, 102, 255, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '399721002',
        schemeDesignator: 'SCT',
        meaning: 'Tumor infiltration by lymphocytes present'
      },
      findingCategory: {
        value: '395557000',
        schemeDesignator: 'SCT',
        meaning: 'Tumor finding'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [51, 204, 204, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    },
    {
      finding: {
        value: '47973001',
        schemeDesignator: 'SCT',
        meaning: 'Artifact'
      },
      geometryTypes: ['polygon', 'freehandpolygon'],
      style: {
        stroke: {
          color: [255, 80, 80, 1],
          width: 2
        },
        fill: {
          color: [255, 255, 255, 0.2]
        }
      }
    }
  ]
}
"""%(gcpClientID)

with open("synlab-slim-viewer/public/config/example.js", "w") as f:
    f.write(settings)

## Update SLIM's Firebase config to deploy configuration



In [9]:
settings_firebase = """
{
  "hosting": {
    "public": "build",
    "ignore": [
      "firebase.json",
      "**/.*",
      "**/node_modules/**"
    ],
    "rewrites": [
      {
        "source": "**",
        "destination": "/index.html"
      }
    ]
  }
}
"""

with open("synlab-slim-viewer/firebase.json", "w") as f:
  f.write(settings_firebase)

## Build SLIM



In [ ]:
!cd synlab-slim-viewer && REACT_APP_CONFIG=example PUBLIC_URL='/' yarn build

yarn run v1.22.22
$ craco build
(node:10793) [DEP0176] DeprecationWarning: fs.F_OK is deprecated, use fs.constants.F_OK instead
(Use `node --trace-deprecation ...` to show where the warning was created)
Creating an optimized production build...
Browserslist: caniuse-lite is outdated. Please run:
  npx update-browserslist-db@latest
  Why you should do it regularly: https://github.com/browserslist/update-db#readme


## Login and initialize Firebase configuration

  

> **User interaction required**: In the next step we login to Firebase and configure the setup of Firebase on the VM. When prompted regarding error reporting information, double click next to the prompt, and type your selection. Next, follow the authorization URL to grant the necessary permissions to Firebase, then double-click next to the "Paste authorization code", paste the code and hit "Enter" to complete the process.

In [ ]:
!firebase login --no-localhost

> **User interaction required**: Here you need to login to your Google account where you will be prompted to follow an authorization URL to grant the necessary permissions. Afterwards, we set the default GCP project ID to the one you chose at the beginning of this notebook.   

In [ ]:
!gcloud auth login

In [ ]:
!gcloud config set project $gcpProjectID
!gcloud config get project

> **User interaction required**: You should have created a Firebase project as a prerequisite step. Select a project or modify SLIM's .firebaserc file which sets the default firebase project.

In [ ]:
!firebase projects:list

In [ ]:
!cd synlab-slim-viewer && firebase use $FirebaseProjectName

## Deploy
> Finally, we deploy the app. Do not close the notebook, keep the tab open and return to the Google documentation.

In [ ]:
!cd synlab-slim-viewer && firebase deploy